# SPARQL 1.1 Overview

> Exemples tirés de <https://www.w3.org/TR/sparql11-overview/>

## Exemple

Les informations utilisée dans le _Triple-Store_ sont constituées par le graphe suivant:

```turtle
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/alice#me> a foaf:Person .
<http://example.org/alice#me> foaf:name "Alice" .
<http://example.org/alice#me> foaf:mbox <mailto:alice@example.org> .
<http://example.org/alice#me> foaf:knows <http://example.org/bob#me> .
<http://example.org/bob#me> foaf:knows <http://example.org/alice#me> .
<http://example.org/bob#me> foaf:name "Bob" .
<http://example.org/alice#me> foaf:knows <http://example.org/charlie#me> .
<http://example.org/charlie#me> foaf:knows <http://example.org/alice#me> .
<http://example.org/charlie#me> foaf:name "Charlie" .
<http://example.org/alice#me> foaf:knows <http://example.org/snoopy> .
<http://example.org/snoopy> foaf:name "Snoopy"@en .
```

## Première requête

A ce stade, les information sont stockées. Maintenant il suffit de les sélectionner. Un autre point d'accès est choisi (en lecture) pour faire les requêtes: `http://sparql:3030/sparql-overview/sparql`.

In [2]:
%endpoint http://sparql:3030/sparql-overview/sparql

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name (COUNT(?friend) AS ?count)
WHERE { 
    ?person foaf:name ?name . 
    ?person foaf:knows ?friend . 
} GROUP BY ?person ?name

Endpoint set to: http://sparql:3030/sparql-overview/sparql

name,count
Alice,3
Charlie,1
Bob,1


## Formats

Les données peuvent être exportées en plusieurs formats

### Format XML

In [3]:
%display raw

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name (COUNT(?friend) AS ?count)
WHERE { 
    ?person foaf:name ?name . 
    ?person foaf:knows ?friend . 
} GROUP BY ?person ?name

Display: raw

<?xml version="1.0"?>
<sparql xmlns="http://www.w3.org/2005/sparql-results#">
  <head>
    <variable name="name"/>
    <variable name="count"/>
  </head>
  <results>
    <result>
      <binding name="name">
        <literal>Alice</literal>
      </binding>
      <binding name="count">
        <literal datatype="http://www.w3.org/2001/XMLSchema#integer">3</literal>
      </binding>
    </result>
    <result>
      <binding name="name">
        <literal>Charlie</literal>
      </binding>
      <binding name="count">
        <literal datatype="http://www.w3.org/2001/XMLSchema#integer">1</literal>
      </binding>
    </result>
    <result>
      <binding name="name">
        <literal>Bob</literal>
      </binding>
      <binding name="count">
        <literal datatype="http://www.w3.org/2001/XMLSchema#integer">1</literal>
      </binding>
    </result>
  </results>
</sparql>


### Format JSON

In [4]:
%format json
%display RAW

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name (COUNT(?friend) AS ?count)
WHERE { 
    ?person foaf:name ?name . 
    ?person foaf:knows ?friend . 
} GROUP BY ?person ?name

Return format: JSON
Display: raw

{
  "head": {
    "vars": [
      "name",
      "count"
    ]
  },
  "results": {
    "bindings": [
      {
        "count": {
          "datatype": "http://www.w3.org/2001/XMLSchema#integer",
          "type": "literal",
          "value": "3"
        },
        "name": {
          "type": "literal",
          "value": "Alice"
        }
      },
      {
        "count": {
          "datatype": "http://www.w3.org/2001/XMLSchema#integer",
          "type": "literal",
          "value": "1"
        },
        "name": {
          "type": "literal",
          "value": "Charlie"
        }
      },
      {
        "count": {
          "datatype": "http://www.w3.org/2001/XMLSchema#integer",
          "type": "literal",
          "value": "1"
        },
        "name": {
          "type": "literal",
          "value": "Bob"
        }
      }
    ]
  }
}


---

## Requêtes Fédérées

Certaines sous-requêtes peuvent être déléguées à d'autres points d'accès sparql

In [5]:
%format any
%display table

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name
WHERE {
    <http://example.org/alice#me> foaf:knows [
        foaf:name ?name
    ] .
    SERVICE <http://dbpedia.org/sparql> { 
        <http://dbpedia.org/resource/Snoopy> foaf:name ?name 
    }
}

Return format: ANY
Display: table

name
Snoopy



---

## Raisonnement

Comme `foaf:name` est `rdfs:subPropertyOf` de `rdfs:label` _(et que le raisonnement est activé dans la configuration du stockage)_:

In [ ]:
%endpoint http://sparql:3030/sparql-overview/sparql

prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?person ?label 
WHERE { 
    ?person a foaf:Person ;
              rdfs:label ?label 
}